In [ ]:
!pip install pymysql

In [ ]:
import pandas as pd
import numpy as np
import pymysql

# mysql 연결 코드 삽입


In [ ]:
# 파일 로드
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1eLMWxjK6ADHMRO3aEmJUStDWa7P4H8Zj" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1eLMWxjK6ADHMRO3aEmJUStDWa7P4H8Zj" -o daily_naver_news.csv

In [ ]:
import pandas as pd
df = pd.read_csv('daily_naver_news.csv')
df

In [ ]:
#형태소 분석기 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
from konlpy.tag import Mecab

In [ ]:
tokenizer = Mecab()

# 정규 표현식을 통해서 한국어 제외 문자들을 제거
df['content'] = df['content'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

In [ ]:
df

In [ ]:
news_data = df.content

In [ ]:
# 결측값 삭제
news_data.dropna(inplace=True)

In [ ]:
# 다소 시간 소요
result = news_data.apply(tokenizer.morphs)

In [ ]:
idx = result.index
for i in idx:
  result[i].insert(0,i)
print(result)

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(result, size=100, window=5, min_count=5, workers=4, sg=0)

In [ ]:
#모델 저장
model.save('word2vec.model')

In [ ]:
from gensim.models import Word2Vec
model_test = Word2Vec.load('word2vec.model')

In [ ]:
keyword_list = ['반도체', '인공지능','프랜차이즈','항공우주','클라우드','블록체인','리테일','패션','언택트','자율주행','뷰티','게임','방탄소년단']

In [ ]:
cursor = conn.cursor(pymysql.cursors.DictCursor)

# 테이블 확인
# sql_input = "SELECT * FROM `theme_stock`;"
# df_this_data = pd.read_sql_query(sql_input, conn)
# print(df_this_data)

# 값 삭제
sql_input = "DELETE FROM `theme_stock` WHERE `id` in (1,2,3,4,5,6,7,8,9,10,11,12,13);"
cursor.execute(sql_input)
conn.commit()

In [ ]:
for i, keyword in enumerate(keyword_list):
  df2 = df[df['content'].str.contains(keyword, na=False)]
  # 방법 2 : 입력받은 키워드와 관련된 단어가 있는 행의 종목들 뽑아냄
  if df2.empty or len(df2.index) < 5:
    model_result = model_test.wv.most_similar(keyword)
    model_data = []
    for j in range(5):
      model_data.append(model_result[j][0])
    print(model_data)
    df_rel = df[df['content'].str.contains(model_data[0], na=False)]
    tmp_rel = df_rel.name.value_counts()
    #print(tmp_rel)
    keyword_stock_rel = tmp_rel.index[:5]
    print(keyword_stock_rel)
    rel_stock = ""
    rel_stock = ','.join(keyword_stock_rel)
    sql = "INSERT INTO `theme_stock` (`id`,`keyword_name`,`related_items`) VALUES (%s, %s, %s)"
    val = (i+1, keyword,rel_stock)
    cursor.execute(sql, val)

 # 방법1 : content에 '키워드'가 있는 행만 뽑아내기
  else:
    tmp = df2.name.value_counts()
    #print(tmp)
    keyword_stock = tmp.index[:5]
    #print(keyword_stock)
    rel_stock = ""
    rel_stock = ','.join(keyword_stock)
    print(rel_stock)
    sql = "INSERT INTO `theme_stock` (`id`,`keyword_name`,`related_items`) VALUES (%s, %s, %s)"
    val = (i+1, keyword, rel_stock)
    cursor.execute(sql, val)
    conn.commit()